In [25]:
import pandas as pd
from nltk.stem import WordNetLemmatizer
import contractions
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

STOP_WORDS = set(stopwords.words('english'))

In [26]:
# Load the data
df1 = pd.read_csv('articles1.csv', index_col=0)
df2 = pd.read_csv('articles2.csv', index_col=0)
df3 = pd.read_csv('articles3.csv', index_col=0)

# Concatenate the data files
df = pd.concat([df1, df2, df3])

In [27]:
# Drop unnecessary columns
df = df.drop(columns=['title', 'date', 'year', 'month', 'url'])

# Drop rows with NaN publication or author
df = df.dropna(subset=['publication', 'author'], how='any').reset_index()

In [28]:
def clean_content(row):
    """
    For each row in the data, clean the content of the article
    by removing contractions, lowercasing the text, removing special characters,
    removing stopwords, and lemmatizing the words.
    :param row: pd.Series
    """
    print(f'{row.name}\r', end='')
    content = row['content']
    
    # Remove contractions
    content = contractions.fix(content)
    
    # Lowercase the text
    content = content.lower()
    
    # Remove special characters
    content = content.replace('\n', ' ').replace('\r', ' ').replace('\t', ' ')
    
    # Remove stopwords
    content = ' '.join([word for word in content.split() if word not in STOP_WORDS])
    
    # Lemmatize the words
    lemmatizer = WordNetLemmatizer()
    content = ' '.join([lemmatizer.lemmatize(word) for word in content.split()])
    
    row['tokens'] = word_tokenize(content)
    
    return row

In [29]:
# Apply the cleaning function to the data
df = df.apply(clean_content, axis=1)

In [30]:
df.drop('index', axis=1, inplace=True)

In [31]:
df.to_pickle('cleaned_articles.pkl')